In [5]:
!pip install streamlit # Install the streamlit package


import streamlit as st
import base64
from io import BytesIO


class Nota:
    def __init__(self, titulo, contenido=""):
        self.titulo = titulo
        self.contenido = contenido

    def mostrar(self):
        return f"Título: {self.titulo}\nContenido: {self.contenido}"

    def obtener_tipo(self):
        return "Nota Básica"


class NotaTexto(Nota):
    def __init__(self, titulo, contenido, formato="normal"):
        super().__init__(titulo, contenido)
        self.formato = formato

    def mostrar(self):
        contenido = self.contenido
        if self.formato == "mayúsculas":
            contenido = self.contenido.upper()
        elif self.formato == "minúsculas":
            contenido = self.contenido.lower()
        return f"Título: {self.titulo}\nContenido: {contenido}"

    def obtener_tipo(self):
        return "Nota de Texto"

class NotaLista(Nota):
    def __init__(self, titulo, elementos=None):
        super().__init__(titulo)
        self.elementos = [] if elementos is None else elementos

    def mostrar(self):
        if not self.elementos:
            return f"Título: {self.titulo}\nLista vacía"

        elementos_texto = ""
        for elem in self.elementos:
            elementos_texto += f"- {elem}\n"

        return f"Título: {self.titulo}\nElementos:\n{elementos_texto}"

    def obtener_tipo(self):
        return "Nota de Lista"

class NotaImagen(Nota):
    def __init__(self, titulo, descripcion, imagen=None):
        super().__init__(titulo, descripcion)
        self.imagen = imagen

    def mostrar(self):
        return f"Título: {self.titulo}\nDescripción: {self.contenido}"

    def obtener_tipo(self):
        return "Nota con Imagen"


class BlocDeNotas:
    def __init__(self):
        self.notas = []

    def crear_nota(self, tipo, titulo, contenido=None, formato=None, elementos=None, imagen=None):
        if tipo == "texto":
            nota = NotaTexto(titulo, contenido or "", formato)
        elif tipo == "lista":
            nota = NotaLista(titulo, elementos)
        elif tipo == "imagen":
            nota = NotaImagen(titulo, contenido or "", imagen)
        else:
            nota = Nota(titulo, contenido or "")

        self.notas.append(nota)
        return nota

    def buscar_por_titulo(self, titulo):
        resultados = []
        for i, nota in enumerate(self.notas):
            if titulo.lower() in nota.titulo.lower():
                resultados.append((i, nota))
        return resultados

    def eliminar_nota(self, indice):
        if 0 <= indice < len(self.notas):
            self.notas.pop(indice)
            return True
        return False


def main():
    st.title("Bloc de Notas")


    if 'bloc_de_notas' not in st.session_state:
        st.session_state.bloc_de_notas = BlocDeNotas()

    if 'elementos_temp' not in st.session_state:
        st.session_state.elementos_temp = []


    st.sidebar.header("Crear Nota")
    tipo_nota = st.sidebar.selectbox("Tipo de Nota", ["texto", "lista", "imagen"])
    titulo = st.sidebar.text_input("Título")


    if tipo_nota == "texto":
        contenido = st.sidebar.text_area("Contenido")
        formato = st.sidebar.selectbox("Formato", ["normal", "mayúsculas", "minúsculas"])

        if st.sidebar.button("Crear Nota de Texto"):
            if titulo:
                st.session_state.bloc_de_notas.crear_nota("texto", titulo, contenido, formato)
                st.sidebar.success(f"Nota creada: {titulo}")
                st.experimental_rerun()

    elif tipo_nota == "lista":
        nuevo_elemento = st.sidebar.text_input("Nuevo elemento")

        col1, col2 = st.sidebar.columns(2)
        with col1:
            if st.button("Añadir elemento"):
                if nuevo_elemento:
                    st.session_state.elementos_temp.append(nuevo_elemento)

        with col2:
            if st.button("Limpiar elementos"):
                st.session_state.elementos_temp = []

        if st.session_state.elementos_temp:
            st.sidebar.write("Elementos actuales:")
            for i, elem in enumerate(st.session_state.elementos_temp):
                st.sidebar.write(f"{i+1}. {elem}")

        if st.sidebar.button("Crear Nota de Lista"):
            if titulo:
                elementos = st.session_state.elementos_temp.copy()
                st.session_state.bloc_de_notas.crear_nota("lista", titulo, elementos=elementos)
                st.session_state.elementos_temp = []
                st.sidebar.success(f"Nota de lista creada: {titulo}")
                st.experimental_rerun()

    elif tipo_nota == "imagen":
        descripcion = st.sidebar.text_area("Descripción")
        imagen_subida = st.sidebar.file_uploader("Subir imagen", type=["png", "jpg", "jpeg"])

        if st.sidebar.button("Crear Nota con Imagen"):
            if titulo and imagen_subida is not None:
                try:
                    imagen_bytes = imagen_subida.getvalue()
                    imagen_b64 = base64.b64encode(imagen_bytes).decode()
                    st.session_state.bloc_de_notas.crear_nota("imagen", titulo, descripcion, imagen=imagen_b64)
                    st.sidebar.success(f"Nota con imagen creada: {titulo}")
                    st.experimental_rerun()
                except Exception as e:
                    st.sidebar.error(f"Error al procesar la imagen: {str(e)}")
            else:
                st.sidebar.warning("Se requiere título e imagen")


    st.header("Mis Notas")


    busqueda = st.text_input("Buscar por título")
    if st.button("Mostrar todas las notas"):
        busqueda = ""


    if busqueda != "":
        resultados = st.session_state.bloc_de_notas.buscar_por_titulo(busqueda)
    else:
        resultados = [(i, nota) for i, nota in enumerate(st.session_state.bloc_de_notas.notas)]

    if not resultados:
        st.info("No hay notas para mostrar")


    for i, nota in resultados:
        with st.expander(f"{nota.titulo} ({nota.obtener_tipo()})"):
            st.write(nota.mostrar())


            if isinstance(nota, NotaImagen) and nota.imagen:
                try:
                    imagen_bytes = base64.b64decode(nota.imagen)
                    st.image(BytesIO(imagen_bytes), caption=nota.titulo)
                except Exception:
                    st.error("No se pudo cargar la imagen")


            if st.button(f"Eliminar nota", key=f"eliminar_{i}"):
                st.session_state.bloc_de_notas.eliminar_nota(i)
                st.success(f"Nota '{nota.titulo}' eliminada")
                st.experimental_rerun()

if __name__ == "__main__":
    main()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


2025-03-31 05:05:58.983 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 05:05:59.111 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-03-31 05:05:59.113 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 05:05:59.115 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 05:05:59.117 Session state does not function when running a script without `streamlit run`
2025-03-31 05:05:59.119 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 05:05:59.120 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 05:05:59.121 Thread 'MainThread': missing ScriptRunContext! 